In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import griddata, interp1d
from torch.utils.data import DataLoader
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import sklearn.model_selection as ms
from IPython.core.display import display, HTML
np.set_printoptions(precision=6, suppress=True)
from pprz_data.pprz_data import DATA
import torch
import torch.nn as nn
from torch.autograd import Variable

In [5]:
def assign_fault(df):
    df = df.assign(fault=0)
    cond1 = (df['add1'] > 0.005) | (df['add1'] < -0.005)
    cond2 = (df['add2'] > 0.005) | (df['add2'] < -0.005)
    cond3 = (df['m1'] < 1.0) | (df['m2'] < 1.0)
    cond = cond1 | cond2 | cond3
    df.loc[cond,'fault'] = 1
#     cond4 = (df['mode'] == 2.0)
#     df = df[cond4]
    return df

In [6]:
ac_id='20'
filename = "/home/jovyan/code/HDMTL/data/12_07_2020_Faulty_Daredevil_onboard_log/20_07_12__06_58_20_SD.data"
sd12 = DATA(filename, ac_id, data_type="flight", sample_period=0.1)

df_12 = sd12.get_labelled_data()
filename = "/home/jovyan/code/HDMTL/data/13_07_2020_Faulty_Daredevil/20_07_13__08_24_59_SD.data"
sd13 = DATA(filename, ac_id, data_type="flight", sample_period=0.1)
df_13 = sd13.get_labelled_data()
df_12 = assign_fault(df_12)
df_13 = assign_fault(df_13)


               phi       psi     theta
time                                  
34.641   -0.007491 -3.141060 -0.070786
34.691   -0.007470 -3.141047 -0.070755
34.741   -0.007450 -3.141038 -0.070722
34.791   -0.007423 -3.141042 -0.070695
34.841   -0.007420 -3.141048 -0.070673
...            ...       ...       ...
2646.291 -0.029659 -3.065667 -0.080085
2646.341 -0.029708 -3.065736 -0.080151
2646.391 -0.029682 -3.065822 -0.080144
2646.441 -0.029639 -3.065858 -0.080094
2646.491 -0.029579 -3.065944 -0.080001

[52238 rows x 3 columns]
          mode
time          
34.891     2.0
35.891     2.0
36.891     2.0
37.891     2.0
38.891     2.0
...        ...
2641.891   0.0
2642.891   0.0
2643.891   0.0
2644.891   0.0
2645.891   0.0

[2612 rows x 1 columns]
                Ax        Ay        Az
time                                  
34.611    0.443359  0.044922 -9.727539
34.631    0.443359  0.054688 -9.715820
34.651    0.448242  0.049805 -9.715820
34.671    0.443359  0.049805 -9.735352
34.691    0.4

                Ax        Ay        Az
time                                  
34.632    0.443359  0.171875 -9.668945
34.652    0.448242  0.137695 -9.723633
34.672    0.469727  0.152344 -9.675781
34.692    0.477539  0.140625 -9.673828
34.712    0.458008  0.125977 -9.690430
...            ...       ...       ...
2998.912  0.482422  0.051758 -9.526367
2998.932  0.415039  0.076172 -9.632812
2998.952  0.479492  0.155273 -9.491211
2998.972  0.500977  0.236328 -9.740234
2998.992  0.551758  0.236328 -9.943359

[148219 rows x 3 columns]
                east        north  course       alt   vel  climb
time                                                            
34.712    36028211.0  481360971.0     0.0  186123.0  15.0   10.0
34.912    36028212.0  481360969.0     0.0  186121.0   9.0    9.0
35.112    36028211.0  481360970.0     0.0  186145.0   5.0   17.0
35.312    36028209.0  481360971.0     0.0  186154.0  20.0    1.0
35.512    36028207.0  481360971.0     0.0  186132.0  16.0   11.0
...        

In [7]:

def add_time_history_1(X,y,n_step=3,t_slide=1):

    
    time_len= X.shape[0] #-n_step        
    column_len = X.shape[1]        
    P=len(np.arange(n_step,time_len,t_slide))    
    xx=np.zeros((P+n_step,column_len*n_step))
    
    for i in range(n_step,P+n_step):
        for j in range(n_step):
            xx[i,j*column_len:(j+1)*column_len] = X[t_slide*i-t_slide*n_step+j]  

    xx = xx[n_step:X.shape[0],:]        
      
    yy = y[[np.arange(n_step,time_len,t_slide)-1]]  
    print(xx.shape)
    return xx,yy


from sklearn.preprocessing import MinMaxScaler
class Time():
    def __init__(self, start, end):
        self.start = start
        self.end = end

In [9]:
# A1
seq_len, input_dim = 128, 20
num_heads, hidden_dim = 8, 128
dropout = 0.1
class MultiHeadAttention2(nn.Module):
    def __init__(self, input_dim, num_heads, hidden_dim, hidden ,dropout):
        super(MultiHeadAttention2, self).__init__()
        self.input_dim = input_dim
        self.num_heads = num_heads
        self.hidden_dim = hidden_dim
        self.dropout = dropout
        
        self.query = nn.Linear(input_dim, hidden_dim)
        self.key = nn.Linear(input_dim, hidden_dim)
        self.value = nn.Linear(input_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, input_dim)
        self.fc1 = nn.Linear(hidden, input_dim)
        self.dropout_layer = nn.Dropout(dropout)
      
    def forward(self, x, mask=None):


        
        batch_size, seq_len, input_dim = x.size()        
        # 计算 Q、K、V
        query = self.query(x).view(batch_size, seq_len, self.num_heads, -1).transpose(1, 2)
        key = self.key(x).view(batch_size, seq_len, self.num_heads, -1).transpose(1, 2)
        value = self.value(x).view(batch_size, seq_len, self.num_heads, -1).transpose(1, 2)       

        dot_product = torch.matmul(query, key.transpose(-1, -2)) / ((self.hidden_dim/num_heads)** 0.5)
        attention_scores = torch.softmax(dot_product, dim=-1)      
        attention_scores = self.dropout_layer(attention_scores)

        weighted_sum = torch.matmul(attention_scores, value).transpose(1, 2).contiguous().view(batch_size, seq_len, -1)


        return weighted_sum
    
    
import torch.nn.utils as utils 

class ASU(nn.Module):
    def __init__(self,size,batch_size,size2, hidden):
        super(ASU, self).__init__()        
        self.matrix1 = utils.weight_norm(nn.Linear(size,size), name='weight')
        self.matrix2 = utils.weight_norm(nn.Linear(size,size), name='weight')
#         self.matrix1 = nn.Linear(size,size)
#         self.matrix2 = nn.Linear(size,size)
        
        self.Attention = MultiHeadAttention2(size2,num_heads,size2, hidden, dropout)
        
    def forward(self, input1, input2):

        input_reshaped = torch.cat((input1.transpose(1, 2), input2), dim=-1)



        output_reshaped0 = self.matrix1(input_reshaped)
        output_reshaped0 = self.matrix2(output_reshaped0)

        
        output_reshaped=self.Attention(output_reshaped0).view(output_reshaped0.shape)

        
        output1 = output_reshaped[:, :,:input2.shape[2]].reshape(input1.shape)
        
        output2 = output_reshaped[:,: ,input2.shape[2]:].view(input2.shape)
        
        
        return output1, output2,self.matrix1
        

In [10]:
INPUT_SIZE=12
class HDMTLNet(nn.Module):
    def __init__(self,output_dim,ASU_enabled):
        super(HDMTLNet, self).__init__()
        self.conv1_1 = nn.Sequential(
            nn.Conv1d(20, 128, 3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool1d(20),
        )
        
        self.conv2_2 = nn.Sequential(
            nn.Conv1d(128, 256, 3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
        )                 
        self.conv3_3 = nn.Sequential(
            nn.Conv1d(256, 512, 3, padding=1),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool1d(1),
        )
        

        
        self.rnn1_1 = nn.LSTM(         
            input_size=INPUT_SIZE,
            hidden_size=128,        
            num_layers=1,           
            batch_first=True,       

        )
        
        self.rnn2_2 = nn.LSTM(        
            input_size=128,
            hidden_size=256,         
            num_layers=1,           
            batch_first=True,       

        )
        self.rnn3_3 = nn.LSTM(         
            input_size=256,
            hidden_size=512,         
            num_layers=1,           
            batch_first=True,       

        )
                            
       
        self.Batch1_1 = nn.Sequential(
            nn.BatchNorm1d(20),
            nn.ReLU(inplace=True),
        )
        self.Batch2_2 = nn.Sequential(
            nn.BatchNorm1d(20),
            nn.ReLU(inplace=True),
        )
        
        self.Batch3_3 = nn.Sequential(
            nn.BatchNorm1d(20),
            nn.ReLU(inplace=True),
        )  
        
        self.layer1 = nn.Sequential(
            nn.Linear(512, 256),
            nn.Linear(256, 256),
            nn.LeakyReLU(inplace=True),
            nn.Linear(256, output_dim),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.Linear(256, 256),
            nn.LeakyReLU(inplace=True),
            nn.Linear(256, output_dim),
        )
        

        self.ASU_enabled = ASU_enabled


        self.ASU1 = ASU(256,batch_size,256,384)
        self.ASU2 = ASU(512,batch_size,512,640)
        self.ASU3 = ASU(1024,batch_size,1024,1152)

    def forward(self, x,epoch):
        cross_start=0
        conv1_1 = self.conv1_1(x)
        rnn1_1,(h_n, h_c) = self.rnn1_1(x, None)
        rnn1_1=self.Batch1_1(rnn1_1)
        if self.ASU_enabled:
            if epoch > cross_start:
                crossC1_1, crossR1_1,matrix = self.ASU1(conv1_1, rnn1_1)
            else:
                crossC1_1, crossR1_1 = conv1_1, rnn1_1
        else:
            crossC1_1, crossR1_1 = conv1_1, rnn1_1
        conv2_2 = self.conv2_2(crossC1_1)

        rnn2_2,(h_n, h_c) = self.rnn2_2(crossR1_1, None)
    
        rnn2_2=self.Batch2_2(rnn2_2)    

        if self.ASU_enabled:
            if epoch > cross_start:
                crossC2_2, crossR2_2,matrix = self.ASU2(conv2_2, rnn2_2)
            else:
                crossC2_2, crossR2_2 = conv2_2, rnn2_2
        else:
            crossC2_2, crossR2_2 = conv2_2, rnn2_2
            
        conv3_3 = self.conv3_3(crossC2_2)

        rnn3_3,(h_n, h_c) = self.rnn3_3(crossR2_2, None)
        rnn3_3=self.Batch3_3(rnn3_3)    

        rnn3_3 = rnn3_3[:, -1, :].reshape(batch_size,1,-1)

        if self.ASU_enabled:
            if epoch > cross_start:

                
                crossC3_3, crossR3_3,matrix = self.ASU3(conv3_3, rnn3_3)
            else:
                
                crossC3_3, crossR3_3 = conv3_3, rnn3_3
                
        else:
            crossC3_3, crossR3_3 = conv3_3, rnn3_3            

        fc2_1 = self.layer1(crossC3_3.squeeze())        
        
        fc2_2 = self.layer2(crossR3_3.squeeze())

        return fc2_1, fc2_2

In [3]:

def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            print("can't create directory '{}''".format(path))
            
            exit(1)
            
def SampleProduction(df, time=None, features=['airspeed', 'phi','psi','theta','Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz', 'C1', 'C2'], targets=['nominal', 'faulty']):
    # Create the Feature and Label List
    columns=features 
    if time==None:
        flight = df.copy()
    else:
        flight = df[time.start:time.end].copy()                                                                                                                     
    X_pre = flight[columns].values # Features          
    y = flight.fault.values   # Labels
    X_sd,y_sd = add_time_history_1(X_pre,y,n_step= 20)
    
    X_train, X_test, y_train, y_test = ms.train_test_split(X_sd, y_sd, test_size=0.1, random_state=42, stratify=y_sd)
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train.reshape(-1,20,12), X_test.reshape(-1,20,12), y_train, y_test

def Cross_train_and_test(exp_name,exps):
    
    X_train0, X_test0, y_train0, y_test0 = SampleProduction(df_12, time, targets = ['nominal', 'R0.3', 'L0.9', 'L0.8','L0.7', 'L0.6', 'L0.5', 'L0.4', 
                                                                               'L0.3']  )
    X_train0=torch.as_tensor(X_train0, dtype=None, device=None)
    
    X_train0 = X_train0.float().cuda()
    
    X_test0=torch.as_tensor(X_test0, dtype=None, device=None)
    X_test0 = X_test0.float().cuda()
    
    train_classes = sorted(list(set(y_train0)))
    
    train_indices = [np.where(y_train0 == i)[0] for i in train_classes]

    

    
    for exp in exps:
        scores1=[]
        scores2=[]
        num = int(exp/len(train_classes))        
        for time_idx in range (times):
            np.random.seed(1)
            a=HDMTLNet(9,ASU_enabled).to(device)
            criterion = nn.CrossEntropyLoss()

            optimizer = optim.Adam([
                {'params': (p for name, p in a.named_parameters() if 'bias'not in name), 'weight_decay': 0.001},
                {'params': (p for name, p in a.named_parameters() if 'bias' in name)},
                ],  lr=0.00001)
    
            settings["save_path"] = "Result_2/%s/size_%s/time_%s/" % (exp_name,exp,time_idx)
            
            mkdir(settings["save_path"])
            
            train_idxs = []
            val_idxs = []
            print("*************%s/size_%s/time_%s/" % (exp_name,exp,time_idx))
            
            for i, c in enumerate(train_classes):
                
                select_idx = train_indices[i][np.random.choice(len(train_indices[i]), num, replace=False)]
                
                split = int(num*0.7)
                
                
                train_idxs.extend(select_idx[:split])
                
                val_idxs.extend(select_idx[split:])
                
            X_val, y_val = X_train0[val_idxs] , y_train0[val_idxs]
            X_train, y_train = X_train0[train_idxs], y_train0[train_idxs], 
            df_train=torch.utils.data.TensorDataset(torch.tensor(X_train),torch.tensor(y_train))
            df_val=torch.utils.data.TensorDataset(torch.tensor(X_val),torch.tensor(y_val))
            df_test=torch.utils.data.TensorDataset(torch.tensor(X_test0),torch.tensor(y_test0))

            df_train = DataLoader(dataset=df_train, batch_size=batch_size, shuffle=True, drop_last=True)
            
            df_val = DataLoader(dataset=df_val, batch_size=batch_size, shuffle=True, drop_last=True)

            df_test = DataLoader(dataset=df_test, batch_size=batch_size, shuffle=True, drop_last=True)            

            save1=0
            save2=0
            for epoch in range(1, Epoch):
                for batch_idx, (data, target) in enumerate(df_train):
                    data, target = data.to(device), target.to(device)
                    a.train()



                    pre1,pre2=a(data,epoch)

                    loss = (criterion(pre1,target)+criterion(pre2,target))

                    a.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
                    if batch_idx % 2000 == 0:
                        a.eval()
                        with torch.no_grad():

                            corrects1 = torch.zeros(1).to(device)
                            corrects2 = torch.zeros(1).to(device)
                            for idx, (data, target ) in enumerate(df_val):
                                data, target  = data.to(device), target.to(device)
                                pre1,pre2=a(data,epoch)
                                _, preds1 = torch.max(pre1, 1)
                                _, preds2 = torch.max(pre2, 1)
                                corrects1 += (preds1 == target).sum()
                                corrects2 += (preds2 == target).sum()
                            acc1 = corrects1.item() / len(df_val.dataset)
                            acc2 = corrects2.item() / len(df_val.dataset)

                            print('***** Eval Result epoch: {:.0f},acc1: {:.4f}, acc2: {:.4f}'.format(epoch,acc1,acc2))  
                            if acc1 > save1:
                                save1=acc1
                                torch.save(a, './saved_plot/model save/HDMTL12.pt')
                                print('model save in 1 **************') 
                                
                            if acc2 > save2:
                                save2=acc2
                                torch.save(a, './saved_plot/model save/HDMTL12.pt')
                                print('model save in 2 **************') 
                                           
            
            path_model='./saved_plot/model save/HDMTL12.pt'
            model =torch.load(path_model)
            model.eval()
            corrects1 = torch.zeros(1).to(device)
            corrects2 = torch.zeros(1).to(device)
            for idx, (data, target ) in enumerate(df_test):
                data, target  = data.to(device), target.to(device)
                pre1,pre2=model(data,epoch)
                _, preds1 = torch.max(pre1, 1)
                _, preds2 = torch.max(pre2, 1)
                corrects1 += (preds1 == target).sum()
                corrects2 += (preds2 == target).sum()
            acc1 = corrects1.item() / len(df_test.dataset)
            acc2 = corrects2.item() / len(df_test.dataset) 
            scores1.append(acc1)
            scores2.append(acc2)
                                           
            print('***** Test Result acc1: {:.4f}, acc2: {:.4f}'.format(acc1,acc2))            

            a =pd.DataFrame(np.array(scores1))
            a.to_csv("Result_2/%s/size_%s/time_%s/scores1.csv" % (exp_name,exp,time_idx),index=True) 
            a =pd.DataFrame(np.array(scores2))
            a.to_csv("Result_2/%s/size_%s/time_%s/scores2.csv" % (exp_name,exp,time_idx),index=True)              
        a =pd.DataFrame(np.array(scores1))
        a.to_csv("Result_2/%s/size_%s/scores1.csv" % (exp_name,exp),index=True) 
        a =pd.DataFrame(np.array(scores2))
        a.to_csv("Result_2/%s/size_%s/scores2.csv" % (exp_name,exp),index=True) 
        
def Cross_analysis(exps_name,exps):
    
    scores_all1 = pd.DataFrame()
    scores_all2 = pd.DataFrame()
    for exp in exps:
        
        file_path = "Result_2/%s/size_%s" % (exp_name,exp)
        
        Result_2_data1 = pd.read_csv("%s/scores1.csv" % (file_path), 
                               sep=',', index_col=0)
        Result_2_data1['exp'] = exp 
        scores_all1 = pd.concat([scores_all1,Result_2_data1],axis=0) 
        
        Result_2_data2 = pd.read_csv("%s/scores2.csv" % (file_path), 
                               sep=',', index_col=0)
        Result_2_data2['exp'] = exp 
        scores_all2 = pd.concat([scores_all2,Result_2_data2],axis=0)   
        
    scores_all1.to_csv("Result_2/%s/scores_all1.csv" % (exp_name), float_format='%.6f', index=True)
    
    scores_all2.to_csv("Result_2/%s/scores_all2.csv" % (exp_name), float_format='%.6f', index=True)


    scores_all1['model'] = 'S'
    scores_all2['model'] = 'T'

    scores_all = pd.concat([scores_all1,scores_all2],axis=0)
    scores_all.to_csv("Result_2/%s/scores_all.csv" % (exp_name), float_format='%.6f', index=True)   
    
    return scores_all


In [16]:
import torch.optim as optim
import errno
batch_size=2
ASU_enabled=True
cross_start=0
Epoch=100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
time=Time(500,1400)
settings = {
  "save_path":""
}
exp_name = "HDMTL Supplementary experiments/HDMTL 12"
# exps = [90,120,150,180,270,360,720,1440]
exps = [90]
times = 10
Cross_train_and_test(exp_name,exps)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  from ipykernel import kernelapp as app


(8992, 240)
*************HDMTL Supplementary experiments/HDMTL 12/size_90/time_0/
[2975, 4777, 799, 7502, 753, 2963, 3934, 3921, 7801, 1896, 7688, 233, 2342, 7209, 886, 2137, 6886, 5923, 227, 6601, 8049, 2378, 5031, 1583, 4098, 953, 243, 3501, 1894, 8061, 5280, 6997, 373, 5133, 7749, 3810, 7215, 2545, 6596, 6873, 7750, 66, 2030, 3599, 1946, 5112, 7572, 2389, 4815, 7852, 4841, 1066, 5265, 5477, 1391, 1346, 5780, 3931, 2463, 1927, 332, 4366]
***** Eval Result epoch: 1,acc1: 0.0000, acc2: 0.0000


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


***** Eval Result epoch: 2,acc1: 0.0000, acc2: 0.0000
***** Eval Result epoch: 3,acc1: 0.5000, acc2: 0.5000


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type HDMTLNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type ASU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type MultiHeadAttention2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


...

***** Eval Result epoch: 1,acc1: 0.0000, acc2: 0.0000
***** Eval Result epoch: 2,acc1: 0.0000, acc2: 0.5000
model save in 2 **************
***** Eval Result epoch: 3,acc1: 0.5000, acc2: 0.5357
model save in 1 **************
model save in 2 **************
***** Eval Result epoch: 4,acc1: 0.5000, acc2: 0.5357
***** Eval Result epoch: 5,acc1: 0.9286, acc2: 0.5000
model save in 1 **************
***** Eval Result epoch: 6,acc1: 0.5000, acc2: 0.5000
***** Eval Result epoch: 7,acc1: 0.5000, acc2: 0.5000
***** Eval Result epoch: 8,acc1: 0.7500, acc2: 0.5714
model save in 2 **************
***** Eval Result epoch: 9,acc1: 0.9286, acc2: 0.9286
model save in 2 **************
***** Eval Result epoch: 10,acc1: 0.9286, acc2: 0.9286
***** Eval Result epoch: 11,acc1: 0.9286, acc2: 0.9286
***** Eval Result epoch: 12,acc1: 0.9286, acc2: 0.9286
***** Eval Result epoch: 13,acc1: 0.9286, acc2: 0.9286
***** Eval Result epoch: 14,acc1: 0.9643, acc2: 0.9643
model save in 1 **************
model save in 2 ******

***** Eval Result epoch: 32,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 33,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 34,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 35,acc1: 1.0000, acc2: 0.9643
***** Eval Result epoch: 36,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 37,acc1: 0.9643, acc2: 0.9643
***** Eval Result epoch: 38,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 39,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 40,acc1: 1.0000, acc2: 0.9643
***** Eval Result epoch: 41,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 42,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 43,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 44,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 45,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 46,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 47,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 48,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 49,acc1: 1.0000, acc2: 1.0000
***** Eval

***** Eval Result epoch: 68,acc1: 0.9643, acc2: 1.0000
***** Eval Result epoch: 69,acc1: 0.9643, acc2: 0.9643
***** Eval Result epoch: 70,acc1: 0.9643, acc2: 0.9643
***** Eval Result epoch: 71,acc1: 0.9643, acc2: 0.9643
***** Eval Result epoch: 72,acc1: 0.9643, acc2: 0.9643
***** Eval Result epoch: 73,acc1: 0.9643, acc2: 0.9643
***** Eval Result epoch: 74,acc1: 0.9643, acc2: 0.9643
***** Eval Result epoch: 75,acc1: 0.9643, acc2: 0.9643
***** Eval Result epoch: 76,acc1: 0.9643, acc2: 0.9643
***** Eval Result epoch: 77,acc1: 0.9643, acc2: 0.9643
***** Eval Result epoch: 78,acc1: 0.9643, acc2: 0.9643
***** Eval Result epoch: 79,acc1: 0.9643, acc2: 1.0000
***** Eval Result epoch: 80,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 81,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 82,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 83,acc1: 0.9643, acc2: 0.9643
***** Eval Result epoch: 84,acc1: 0.9643, acc2: 1.0000
***** Eval Result epoch: 85,acc1: 0.9643, acc2: 0.9643
***** Eval

***** Eval Result epoch: 1,acc1: 0.5000, acc2: 0.0000
model save in 1 **************
***** Eval Result epoch: 2,acc1: 0.5000, acc2: 0.5000
model save in 2 **************
***** Eval Result epoch: 3,acc1: 0.5000, acc2: 0.5357
model save in 2 **************
***** Eval Result epoch: 4,acc1: 0.5000, acc2: 0.5000
***** Eval Result epoch: 5,acc1: 0.5000, acc2: 0.5000
***** Eval Result epoch: 6,acc1: 0.5000, acc2: 0.5000
***** Eval Result epoch: 7,acc1: 0.5000, acc2: 0.5000
***** Eval Result epoch: 8,acc1: 0.9286, acc2: 0.6071
model save in 1 **************
model save in 2 **************
***** Eval Result epoch: 9,acc1: 0.8214, acc2: 0.8214
model save in 2 **************
***** Eval Result epoch: 10,acc1: 0.8214, acc2: 0.9286
model save in 2 **************
***** Eval Result epoch: 11,acc1: 0.8929, acc2: 0.8929
***** Eval Result epoch: 12,acc1: 0.8929, acc2: 0.8929
***** Eval Result epoch: 13,acc1: 0.9286, acc2: 0.9286
***** Eval Result epoch: 14,acc1: 0.9643, acc2: 0.9643
model save in 1 ******

***** Eval Result epoch: 31,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 32,acc1: 0.9286, acc2: 0.9286
***** Eval Result epoch: 33,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 34,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 35,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 36,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 37,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 38,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 39,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 40,acc1: 1.0000, acc2: 0.9643
***** Eval Result epoch: 41,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 42,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 43,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 44,acc1: 0.9286, acc2: 0.9286
***** Eval Result epoch: 45,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 46,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 47,acc1: 1.0000, acc2: 1.0000
***** Eval Result epoch: 48,acc1: 1.0000, acc2: 1.0000
***** Eval

In [1]:
scores_all = Cross_analysis(exp_name,exps)
scores_all_mean = scores_all.groupby(['model','exp']).mean()
scores_all_std = scores_all.groupby(['model','exp']).std()
scores_all_mean.to_csv("Result_2/%s_all_mean.csv" % (exp_name), float_format='%.4f', index=True)
scores_all_std.to_csv("Result_2/%s_all_std.csv" % (exp_name), float_format='%.4f', index=True)
# scores_all_std = scores_all_std.astype(tf.float32)
scores_all_mean,scores_all_std

NameError: name 'Cross_analysis' is not defined